Lets do this!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#This sends warning messages to the cornfield

import warnings
def ignore_warnings(*args, **kwargs):
    pass
warnings.warn = ignore_warnings

In [ ]:
spray = pd.read_csv("./all/spray.csv")
weather = pd.read_csv("./all/weather.csv")
train = pd.read_csv("./all/train.csv")
test = pd.read_csv("./all/test.csv")

#### EDA

Spray Dataset

In [ ]:
spray.head()

In [ ]:
spray.shape

In [ ]:
spray.dtypes

In [ ]:
spray.isnull().sum()

In [ ]:
spray[spray["Time"].isnull()].head()

Observations:
- Null values are from 1 date, 9/7/2011
- Should potentially change date column into datetime object.

Weather Dataset

In [ ]:
weather.head()

In [ ]:
weather.shape

In [ ]:
weather.dtypes

In [ ]:
# weather.isnull().sum()

In [ ]:
weather["Tmax"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather[weather["Tmax"].apply(type) != int]

In [ ]:
weather["Tmin"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather[weather["Tavg"] == "M"]

In [ ]:
weather[weather["Tavg"] == "T"]

In [ ]:
weather["Tavg"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["Depart"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather[weather["Depart"] == "M"]

In [ ]:
weather[weather["Depart"] == "T"]

In [ ]:
weather["DewPoint"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["WetBulb"].value_counts().sort_values(ascending=False).head()

In [ ]:
# weather[weather["WetBulb"] == "M"]

In [ ]:
# weather[weather["WetBulb"] == "T"]

In [ ]:
weather["Heat"].value_counts().sort_values(ascending=False)

In [ ]:
# weather[weather["Heat"] == "M"]

In [ ]:
weather[weather["Heat"] == "T"]

In [ ]:
weather["Cool"].value_counts().sort_values(ascending=False)

In [ ]:
weather["Sunrise"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["Sunset"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["Depth"].value_counts()

In [ ]:
weather["SnowFall"].value_counts()

In [ ]:
#Drop column
weather["Water1"].value_counts()

In [ ]:
#Drop column
weather["CodeSum"].value_counts().head()

In [ ]:
weather["PrecipTotal"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["StnPressure"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["SeaLevel"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["ResultSpeed"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["ResultDir"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather["AvgSpeed"].value_counts().sort_values(ascending=False).head()

In [ ]:
weather.replace("T", 1e-3, inplace = True)
weather.replace(" T", 1e-3, inplace = True)
weather.replace("  T", 1e-3, inplace = True)

In [ ]:
weather.drop("CodeSum", axis = 1, inplace = True)

In [ ]:
weather.drop("Water1", axis = 1, inplace = True)

In [ ]:
weather.drop("SnowFall", axis = 1, inplace = True)

In [ ]:
weather.drop("Depth", axis = 1, inplace = True)

In [ ]:
weather

In [ ]:
weather_1 = weather[weather['Station']==1]
weather_2 = weather[weather['Station']==2]

In [ ]:
weather_combined = weather_1.merge(weather_2, on='Date', suffixes=("_1", "_2"))

In [ ]:
weather_combined.shape
pd.set_option("display.max_columns", 40)
weather_combined.head()

In [ ]:
weather_combined.drop("Station_1", axis = 1, inplace = True)
weather_combined.drop("Station_2", axis = 1, inplace = True)

In [ ]:
weather_combined["Tavg_2"].replace("M", (weather_combined["Tmax_2"] + weather_combined["Tmin_2"])/2, inplace=True)

In [ ]:
weather_combined["Heat_2"].replace("M", weather_combined["Heat_1"], inplace = True)

In [ ]:
weather_combined["Cool_2"].replace("M", weather_combined["Cool_1"], inplace = True)

In [ ]:
weather_combined["Date"] = pd.to_datetime(weather_combined["Date"])

In [ ]:
weather_combined.drop("Sunrise_2", axis = 1, inplace = True)

In [ ]:
weather_combined.drop("Sunset_2", axis = 1, inplace = True)

In [ ]:
weather_combined.head()

Observations:


Train Dataset

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()

In [ ]:
train["Species"].value_counts()

In [ ]:
train["Address"].value_counts().head()

In [ ]:
train["WnvPresent"].value_counts(normalize = True)

Observations:
- Classes are imbalanced. Will need to balance.
- Should potentially change date column into datetime object.

Test Dataset

In [ ]:
test.head(20)

In [ ]:
test.shape

In [ ]:
test.dtypes

In [ ]:
test["Species"].value_counts()

In [ ]:
test["Address"].value_counts().head()

Observations:
- Does not contain NumMosquitos column; should not be used as a predictor.
- Should potentially change date column into datetime object.
- Not all cats are present in both train and test sets, need to fix before creating dummy variables

#### Spray data edits

In [ ]:
spray.head()

In [ ]:
columns =  ['Time']
spray.drop(columns, inplace=True, axis=1)

In [ ]:
from datetime import datetime
spray['Date'] = pd.DatetimeIndex(spray['Date'])

In [ ]:
spray.head()

#### Spray outliers: show and remove

In [ ]:
import seaborn as sns
sns.boxplot(spray['Latitude'])

In [ ]:
sns.boxplot(spray['Longitude'],color='green')

In [ ]:
spray[spray.Longitude<=-87.9].shape[0]

In [ ]:
spray[spray.Latitude>=42.1].shape[0]

Create filter for Longitude outliers

In [ ]:
long_ol = spray['Longitude']<=-87.9
spray[long_ol].shape

In [ ]:
spray[~long_ol].shape

Remove spray outliers

In [ ]:
spray.shape

In [ ]:
spray = spray[~long_ol]

In [ ]:
spray.shape

#### Add Airport Locations

Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level<br>
Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

In [ ]:
#ohare, midway locations
airport_lat  = [41.995, 41.786]
airport_long = [-87.933,-87.752]

In [ ]:
a_lat = pd.Series(airport_lat)
a_long = pd.Series(airport_long)

#### Map with spray data, and airport locations

In [ ]:
# Code is borrowed from: https://www.kaggle.com/users/213536/vasco/predict-west-nile-virus/west-nile-heatmap

#import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline # Remove this when copying to a script on Kaggle:

mapdata = np.loadtxt('./all/mapdata_copyright_openstreetmap_contributors.txt')
#traps = pd.read_csv('../input/train.csv')[['Date', 'Trap','Longitude', 'Latitude', 'WnvPresent']]

aspect = mapdata.shape[0] * 1.0 / mapdata.shape[1]
lon_lat_box = (-88, -87.5, 41.6, 42.1)

plt.figure(figsize=(10,14))
plt.imshow(mapdata, 
           cmap=plt.get_cmap('gray'), 
           extent=lon_lat_box, 
           aspect=aspect)

locations = train[['Longitude', 'Latitude']].drop_duplicates().values
plt.scatter(locations[:,0], locations[:,1], marker='o',c='k',label = "Mosquito Traps")
plt.scatter(a_long, a_lat, marker = "o",c ='green',label="Airport")
plt.scatter(spray['Longitude'],spray['Latitude'],label="Spray")
plt.legend()

In [ ]:
test_train = pd.concat([test,train],keys = ['test','train'],sort = False)

In [ ]:
test_train.index.levels[0].name = 'og'
test_train.index.levels[1].name = 'pip'

In [ ]:
test_train.columns = [i.lower().replace(' ','_') for i in test_train.columns.get_values()]

In [ ]:
test_train.date=pd.to_datetime(test_train.date, format='%Y-%m-%d')

In [ ]:
test_train.date.head()

In [ ]:
test_train.trap.value_counts().head()

In [ ]:
test_train.info()

In [ ]:
test_train.select_dtypes(include='int')

In [ ]:
test_train = pd.get_dummies(test_train, columns=['species','trap'])

In [ ]:
test_train.info(verbose=True)

In [ ]:
test_train.longitude

In [ ]:
test_train.latitude

I'm going to sugest we don't use Block, Address, and street. They give the same info as lat and lon.  

In [ ]:
def lat_lon_int(x):
    return int(x*1000)

In [ ]:
test_train['lat_int'] = test_train.latitude.apply(lat_lon_int)

In [ ]:
test_train['lon_int'] = test_train.longitude.apply(lat_lon_int)

In [ ]:
def get_month(m):
    return m.month

In [ ]:
test_train['month'] = test_train.date.apply(get_month)

In [ ]:
def get_day(d):
    return d.day

In [ ]:
test_train['day'] = test_train.date.apply(get_day)

In [ ]:
test_train.addressaccuracy.value_counts()

Maybe we should do something about the three values.

In [ ]:
test_train.nummosquitos.value_counts()

In [ ]:
spray.info()

In [ ]:
spray.columns = [i.lower().replace(' ','_') for i in spray.columns.get_values()]

In [ ]:
spray.rename(columns={'latitude': 'latitude_spray', 'longitude': 'longitude_spray'}, inplace=True)

In [ ]:
X_test = pd.DataFrame(test_train.loc['test'])
X_train = pd.DataFrame(test_train.loc['train'])
X_test.shape

In [ ]:
#X_test = X_test.merge(spray, on='date')
X_train = X_train.merge(spray, on='date')

In [ ]:
X_train.shape

In [ ]:
weather_combined.columns = [i.lower().replace(' ','_') for i in weather_combined.columns.get_values()]

In [ ]:
weather_combined = weather_combined.replace('M', np.NaN)
weather_combined = weather_combined.replace('-', np.NaN)
weather_combined = weather_combined.replace('T', np.NaN)
weather_combined = weather_combined.replace(' T', np.NaN)
weather_combined = weather_combined.replace('  T', np.NaN)

In [ ]:
weather_combined.info()

In [ ]:
weather_combined.drop('depart_2',axis = 1,inplace=True)

In [ ]:
weather_combined.head()

In [ ]:
weather_combined.tavg_1=weather_combined.tavg_1.apply(int) #avgspeed_1
weather_combined.depart_1=weather_combined.depart_1.apply(int)
weather_combined.wetbulb_1.fillna(weather_combined.wetbulb_1.median(),inplace=True)
weather_combined.wetbulb_1=weather_combined.wetbulb_1.apply(int)
weather_combined.heat_1=weather_combined.heat_1.apply(int)
weather_combined.cool_1=weather_combined.cool_1.apply(int)
weather_combined.sunrise_1=weather_combined.sunrise_1.apply(int)
weather_combined.sunset_1=weather_combined.sunset_1.apply(int)
weather_combined.preciptotal_1=weather_combined.preciptotal_1.apply(float)
weather_combined.stnpressure_1=weather_combined.stnpressure_1.apply(float)
weather_combined.stnpressure_1.fillna(weather_combined.stnpressure_1.mean(),inplace=True)
weather_combined.sealevel_1=weather_combined.sealevel_1.apply(float)
#weather_combined.avgspeed_1.fillna(weather_combined.avgspeed_1.mean(),inplace=True)
weather_combined.avgspeed_1=weather_combined.avgspeed_1.apply(float)
weather_combined.avgspeed_1.fillna(weather_combined.avgspeed_1.mean(),inplace=True)
weather_combined.tavg_2=weather_combined.tavg_2.apply(int)
weather_combined.wetbulb_2.fillna(weather_combined.wetbulb_2.median(),inplace=True)
weather_combined.wetbulb_2=weather_combined.wetbulb_2.apply(int)
weather_combined.heat_2=weather_combined.heat_2.apply(int)
weather_combined.cool_2=weather_combined.cool_2.apply(int)
weather_combined.preciptotal_2=weather_combined.preciptotal_2.apply(float)
weather_combined.preciptotal_2.fillna(weather_combined.preciptotal_2.mean(),inplace=True)
weather_combined.stnpressure_2=weather_combined.stnpressure_2.apply(float)
weather_combined.stnpressure_2.fillna(weather_combined.stnpressure_2.mean(),inplace=True)
weather_combined.sealevel_2=weather_combined.sealevel_2.apply(float)
weather_combined.sealevel_2.fillna(weather_combined.sealevel_2.mean(),inplace=True)
weather_combined.avgspeed_2=weather_combined.avgspeed_2.apply(float)
weather_combined.avgspeed_2.fillna(weather_combined.avgspeed_2.mean(),inplace=True)

In [ ]:
#X_test = X_test.merge(weather_combined, on='date')
X_train = X_train.merge(weather_combined, on='date')

In [ ]:
X_train.drop('id',axis=1, inplace=True)

In [ ]:
X_train.info(verbose=True, null_counts=True)

In [ ]:
X_test.shape

In [ ]:
X_test = X_test.merge(weather_combined, on='date')
X_test.shape

In [ ]:
X_test.drop(['nummosquitos','wnvpresent'],axis=1,inplace=True)

In [ ]:
X_test.info(verbose=True, null_counts=True)

In [ ]:
#X_train.to_csv('./data/train.csv',index=False)
#X_test.to_csv('./data/test.csv',index=False)

In [ ]:
X_train = pd.read_csv('./data/train.csv')
X_test = pd.read_csv('./data/test.csv')

In [ ]:
df_data = pd.concat([X_test,X_train],sort = False)

In [ ]:
df_data.drop('id', axis =  1, inplace=True) 
df_data.drop(df_data.loc[:,'species_CULEX ERRATICUS':'trap_T903'].columns, axis=1, inplace=True)



In [ ]:
df_data.fillna(-1,inplace=True)

In [ ]:
df_data.date=pd.to_datetime(df_data.date, format='%Y-%m-%d')

In [ ]:
df_data.info(verbose = True)

In [ ]:
train = pd.read_csv("./all/train.csv")
test = pd.read_csv("./all/test.csv")

In [ ]:
df_data2 = pd.concat([test,train],sort = False)

In [ ]:
df_data2.info()

In [ ]:
df_data2.columns = [i.lower().replace(' ','_') for i in df_data2.columns.get_values()]

In [ ]:
df_data2.date=pd.to_datetime(df_data2.date, format='%Y-%m-%d')

In [ ]:
df_data3 = df_data2[['species','trap']].copy()

In [ ]:
df_data3.info()

In [ ]:
del df_data2

In [ ]:
df_data.to_csv('./data/temp.csv',index=False)
df_data3.to_csv('./data/temp2.csv',index=False)

In [ ]:
df_data = pd.read_csv('./data/temp.csv')
df_data3 = pd.read_csv('./data/temp2.csv')

In [ ]:
df_data.info(verbose= True)

In [ ]:
df_data3.info(verbose= True)

In [ ]:
df_data2 = pd.concat([df_data,df_data3],axis = 1,sort = False)

In [ ]:
df_data2.to_csv('./data/graph.csv')

Time to model

In [ ]:
X_train = pd.read_csv('./data/train.csv')
X_test = pd.read_csv('./data/test.csv')

In [ ]:
y_train_wnv = pd.DataFrame(X_train.wnvpresent)
y_train_mos = pd.DataFrame(X_train.nummosquitos)

In [ ]:
X_train.drop(['wnvpresent','nummosquitos'],axis=1,inplace=True)

In [ ]:
X_train.drop(['address','addressnumberandstreet','street'], axis =1,inplace=True)

In [ ]:
X_train.drop('date',axis=1,inplace=True)

In [ ]:
#X_train.date = pd.to_datetime(X_train.date,format='%Y-%m-%d')

In [ ]:
y_train_wnv.wnvpresent = np.where(y_train_wnv.wnvpresent==0.0,0,1)
#y_train_mos.nummosquitos = np.where(y_train.nummosquitos==)

In [ ]:
df_kaggle = pd.DataFrame(X_test.id)

In [ ]:
X_test.drop(['id','date','address','addressnumberandstreet','street'],axis=1,inplace=True)

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_params = {
    'n_neighbors':[1,3,5,9,15,21],
    'weights':['uniform','distance'],
    'metric':['euclidean','manhattan','minkowski'],
    'algorithm':['auto', 'ball_tree', 'kd_tree']
}

knnc_gridsearch = GridSearchCV(KNeighborsClassifier(), knn_params, cv=3, verbose=1, n_jobs=2)

knnc_gridsearch = knnc_gridsearch.fit(X_train, y_train_wnv)

In [ ]:
knnc_gridsearch.best_score_

0.9466971254521226

In [ ]:
knnc_gridsearch.best_params_

{'algorithm': 'auto',
 'metric': 'euclidean',
 'n_neighbors': 21,
 'weights': 'uniform'}

In [ ]:
df_knnc_gs = pd.DataFrame(knnc_gridsearch.cv_results_)
df_knnc_gs.to_csv('./data/knnc_gs_score.csv',index=False)

In [ ]:
df_knnc_gs.head()

In [ ]:
cleo = knnc_gridsearch.predict(X_test)

In [ ]:
df_kaggle['WnvPresent'] = cleo

In [ ]:
df_kaggle.id=df_kaggle.id.apply(int)
#df_kaggle.set_index("id", inplace = True)
df_kaggle.to_csv('./data/knnc_kaggel.csv',index=False)

Kaggel 0.49813

In [ ]:
knnc = KNeighborsClassifier()
model = knnc.fit(X_train,y_train_wnv)
pred_knnc = knnc.predict(X_test)
pred_knnc_train = knnc.predict(X_train)
model.score(X_train, y_train_wnv)

Value for score 0.9507900247477632

Adding Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc_params = {
    'n_estimators':[3,5,10,15,21],
    #'criterion':['mae','mse'],
    'max_features':['auto','sqrt','log2'],
    'max_depth':[3,5,7,11,13,17,19]
}

rfc_gridsearch = GridSearchCV(RandomForestClassifier(), rfc_params, cv=3, verbose=1, n_jobs=2)

rfc_gridsearch = rfc_gridsearch.fit(X_train, y_train_wnv)

In [ ]:
rfc_gridsearch.best_score_

0.9428897772701313

In [ ]:
rfc_gridsearch.best_params_

{'max_depth': 3, 'max_features': 'log2', 'n_estimators': 3}

In [ ]:
df_rfc_gs = pd.DataFrame(rfc_gridsearch.cv_results_)
df_rfc_gs.to_csv('./data/rfc_gs_score.csv',index=False)

In [ ]:
df_rfc_gs.head()

In [ ]:
cleo = rfc_gridsearch.predict(X_test)
df_kaggle['WnvPresent'] = cleo
df_kaggle.id=df_kaggle.id.apply(int)
df_kaggle.to_csv('./data/rfc_kaggel.csv',index=False)

Kaggel 0.50000

In [ ]:
rfc = RandomForestClassifier()
model = rfc.fit(X_train,y_train_wnv)
pred_rfc = rfc.predict(X_test)
pred_rfc_train = rfc.predict(X_train)
model.score(X_train, y_train_wnv)

0.9765848086807538

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgc = XGBClassifier()
model = xgc.fit(X_train, y_train_wnv)
pred_xgc = xgc.predict(X_test)
pred_xgc_train = xgc.predict(X_train)
model.score(X_train, y_train_wnv)

0.9475537787930707

In [ ]:
xgc_params = {
    'max_depth':[3,5,7,11,13,17,19],
    'n_estimators':[3,5,10,15,21],
    'booster':['gbtree', 'gblinear', 'dart']
}

xgc_gridsearch = GridSearchCV(XGBClassifier(), xgc_params, cv=3, verbose=1, n_jobs=2)

xgc_gridsearch = xgc_gridsearch.fit(X_train, y_train_wnv)

In [ ]:
xgc_gridsearch.best_score_

0.9475537787930707

In [ ]:
xgc_gridsearch.best_params_

{'booster': 'gblinear', 'max_depth': 3, 'n_estimators': 3}

In [ ]:
df_xgc_gs = pd.DataFrame(xgc_gridsearch.cv_results_)
df_xgc_gs.to_csv('./data/xgc_gs_score.csv',index=False)

In [ ]:
cleo = xgc_gridsearch.predict(X_test)
df_kaggle['WnvPresent'] = cleo
df_kaggle.id=df_kaggle.id.apply(int)
df_kaggle.to_csv('./data/xgc_kaggel.csv',index=False)

Modeling

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
#throw all object features into a list, except date.
obj_list = []
for i in train.select_dtypes(include='object').columns:
    obj_list.append(i)
obj_list.remove("date")

In [ ]:
obj_list

In [ ]:
#ensure both train and test have all categories to prevent mismatched columns when dummying variables
for col in obj_list:
    train_vals = list(train[col].unique())
    test_vals = list(test[col].unique())
    total_vals = set(train_vals + test_vals)
    train[col] = pd.Categorical(train[col], categories = total_vals)
    test[col] = pd.Categorical(test[col], categories = total_vals)

In [ ]:
#dummy variables
train = pd.get_dummies(train, columns = obj_list)
test = pd.get_dummies(test, columns = obj_list)

In [ ]:
#check to see if the shape looks fine.
print(train.shape)
print(test.shape)

In [ ]:
features = [x for x in train.columns if x not in ["nummosquitos", "wnvpresent", "date"]]
len(features)

In [ ]:
remove = ["id", "date"]
test_features = [x for x in test if x not in remove]
len(test_features)

In [ ]:
df_maj = train[train.wnvpresent == 0]
df_min = train[train.wnvpresent == 1]

In [ ]:
from sklearn.utils import resample

In [ ]:
df_upsample = resample(df_min, replace=True, n_samples=9000, random_state=42)
df_upsample = pd.concat([df_upsample, df_maj])

In [ ]:
df_upsample["wnvpresent"].value_counts(normalize=True)

In [ ]:
X = df_upsample[features]
y = df_upsample["wnvpresent"]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

Logistic Regression

In [ ]:
logreg = LogisticRegression()
ss = StandardScaler()

pipe_logreg = Pipeline([
    ("ss", ss),
    ("logreg", logreg)
])

params = {
    "logreg__penalty" : ["l1", "l2"],
    "logreg__C" : [0.1, 0.5, 1, 5]

}

gs_logreg = GridSearchCV(pipe_logreg, param_grid=params)
gs_logreg.fit(X_train, y_train)

print(gs_logreg.best_params_)
print("CrossValScore", gs_logreg.best_score_)
print(gs_logreg.score(X_test, y_test))

In [ ]:
df_logreg_gs = pd.DataFrame(gs_logreg.cv_results_)
df_logreg_gs.to_csv('./data/logreg_gs_scores.csv',index=False)

In [ ]:
ss1 = StandardScaler()
test_scaled = ss1.fit_transform(test[test_features])
predicts_logreg = gs_logreg.predict(test_scaled)

df_logreg_preds = pd.DataFrame()
ids_1 = test["id"].astype(int)

df_logreg_preds["Id"] = ids_1
df_logreg_preds["WnvPresent"] = predicts_logreg
df_logreg_preds.set_index("Id", inplace = True)

In [ ]:
#For interpretability
logreg1 = LogisticRegression(C = 5, penalty = 'l1')
logreg1.fit(X_train,y_train)
coefs_df_logreg = pd.DataFrame({'feature': features, 'coef_values': logreg1.coef_[0]})

coefs_df_logreg["abs_values"] = coefs_df_logreg["coef_values"].abs()
coefs_df_logreg.sort_values("abs_values", ascending = False).head(20)

In [ ]:
logreg_top = coefs_df_logreg.sort_values("abs_values", ascending = False).head(10)

plt.style.use('fivethirtyeight')
logreg_top.plot(x="feature", y="coef_values", kind="barh", legend=False, title="Logistic Regression Top Predictors")

In [ ]:
# gut check
# df_logreg_preds[df_logreg_preds["WnvPresent"] == 1]

In [ ]:
df_logreg_preds.head()

In [ ]:
df_logreg_preds.to_csv("./data/predicts_logreg.csv")

Extra Trees

In [ ]:
et = ExtraTreesClassifier()

params = {
    "max_depth" : [5, 10, 20],
    "n_estimators" : [10, 20, 40]  
}

gs_et = GridSearchCV(et, param_grid=params)
gs_et.fit(X_train, y_train)

print(gs_et.best_params_)
print("CrossValScore", gs_et.best_score_)
print("Score: ", gs_et.score(X_test, y_test))

In [ ]:
df_et_gs = pd.DataFrame(gs_et.cv_results_)
df_et_gs.to_csv('./data/et_gs_scores.csv',index=False)

In [ ]:
predicts_et = gs_et.predict(test[test_features])

df_et_preds = pd.DataFrame()
ids_2 = test["id"].astype(int)

df_et_preds["Id"] = ids_2
df_et_preds["WnvPresent"] = predicts_et
df_et_preds.set_index("Id", inplace = True)

In [ ]:
#For interpretability
et1 = ExtraTreesClassifier(max_depth=20, n_estimators=20)
et1.fit(X_train, y_train)
features_df_et = pd.DataFrame({'feature': features, 'feature_importances': et1.feature_importances_})
features_df_et.sort_values("feature_importances", ascending = False).head(10)

In [ ]:
et_top = features_df_et.sort_values("feature_importances", ascending = False).head(10)

plt.style.use('fivethirtyeight')
et_top.plot(x="feature", y="feature_importances", kind="barh", legend=False, title="Extra Trees Top Predictors")

In [ ]:
# gut check
# df_et_preds[df_et_preds["WnvPresent"] == 1]

In [ ]:
df_et_preds.head()

In [ ]:
df_et_preds.to_csv("./data/predicts_et.csv")

AdaBoost

In [ ]:
ada = AdaBoostClassifier()

params = {
    "learning_rate" : [0.5, 0.6, 0.7],
    "n_estimators" : [30, 40, 50, 60] 
}

gs_ada = GridSearchCV(ada, param_grid=params)
gs_ada.fit(X_train, y_train)

print(gs_ada.best_params_)
print("CrossValScore", gs_ada.best_score_)
print("Score: ", gs_ada.score(X_test, y_test))

In [ ]:
df_ada_gs = pd.DataFrame(gs_ada.cv_results_)
df_ada_gs.to_csv('./data/ada_gs_scores.csv',index=False)

In [ ]:
predicts_ada = gs_ada.predict(test[test_features])

df_ada_preds = pd.DataFrame()
ids_3 = test["id"].astype(int)

df_ada_preds["Id"] = ids_3
df_ada_preds["WnvPresent"] = predicts_ada
df_ada_preds.set_index("Id", inplace = True)

In [ ]:
test["wnv_predict"] = predicts_ada

In [ ]:
test.to_csv("./data/test_predicts_ada.csv")

In [ ]:
#For interpretability
ada1 = AdaBoostClassifier(learning_rate=0.7, n_estimators=60)
ada1.fit(X_train, y_train)
features_df_ada = pd.DataFrame({'feature': features, 'feature_importances': ada1.feature_importances_})
features_df_ada.sort_values("feature_importances", ascending = False).head(10)

In [ ]:
ada_top = features_df_ada.sort_values("feature_importances", ascending = False).head(10)

plt.style.use('fivethirtyeight')
ada_top.plot(x="feature", y="feature_importances", kind="barh", legend=False, title="AdaBoost Top Predictors")

In [ ]:
# gut check
# df_ada_preds[df_ada_preds["WnvPresent"] == 1]

In [ ]:
df_ada_preds.head()

In [ ]:
df_ada_preds.to_csv("./data/predicts_ada.csv")

Naive Bayes - Multinomial

In [ ]:
gausnb = GaussianNB()
ss = StandardScaler()

pipe_gausnb = Pipeline([
    ("ss", ss),
    ("gausnb", gausnb)
])


pipe_gausnb.fit(X_train, y_train)

print("CrossValScore", pipe_gausnb.score(X_train, y_train))
print("Score: ", pipe_gausnb.score(X_test, y_test))

In [ ]:
predicts_gausnb = pipe_gausnb.predict(test[test_features])

df_gausnb_preds = pd.DataFrame()
ids_4 = test["id"].astype(int)

df_gausnb_preds["Id"] = ids_4
df_gausnb_preds["WnvPresent"] = predicts_gausnb
df_gausnb_preds.set_index("Id", inplace = True)

In [ ]:
# gut check
# df_gausnb_preds[df_gausnb_preds["WnvPresent"] == 1]

In [ ]:
df_gausnb_preds.head()

In [ ]:
df_gausnb_preds.to_csv("./data/predicts_gausnb.csv")

Kaggel 0.50000

In [ ]:
test_train.longitude.unique()

In [ ]:
df_graph = pd.read_csv('./data/graph.csv')

In [ ]:
df_graph=df_graph[df_graph['wnvpresent'] != -1]

In [ ]:
df_graph.drop(['Unnamed: 0','address','street','addressnumberandstreet','addressaccuracy','lat_int',
              'lon_int'],axis = 1, inplace = True)

In [ ]:
df_graph.drop(df_graph.loc[:,'tmax_1':'avgspeed_2'].columns, axis=1, inplace=True)


In [ ]:
from datetime import datetime

In [ ]:
def get_year(y):
    return y.year

In [ ]:
df_graph.date=pd.to_datetime(df_graph.date, format='%Y-%m-%d')

In [ ]:
df_graph['year'] = df_graph.date.apply(get_year)

In [ ]:
df_graph2 = pd.DataFrame(df_graph.groupby(['month','year','trap'],as_index=False)['nummosquitos'].sum())

In [ ]:
df_graph2.shape

In [ ]:
df_graph4 = pd.DataFrame(df_graph.groupby(['month','year','trap'],as_index=False)['wnvpresent'].sum())
df_graph4.shape

In [ ]:
df_graph2['wnvpresent']=df_graph4.wnvpresent
df_graph2[df_graph2['wnvpresent']>1].head()

In [ ]:
df_graph4 = pd.DataFrame(df_graph.groupby(['month','year','trap'],as_index=False)['latitude'].min())
df_graph4.head()

In [ ]:
df_graph2['latitude']=df_graph4.latitude

In [ ]:
df_graph4 = pd.DataFrame(df_graph.groupby(['month','year','trap'],as_index=False)['longitude'].min())
df_graph2['longitude']=df_graph4.longitude
df_graph2.head()

In [ ]:
df_graph4 = pd.DataFrame(df_graph.groupby(['month','year','trap'],as_index=False)['species'].apply(lambda x: "%s" % ', '.join(x)))
df_graph4.reset_index(inplace=True)
df_graph4.rename(columns={0:'species'},inplace=True)
#df_graph4.head()
df_graph2['species']=df_graph4['species']
df_graph2.head()

In [ ]:
df_graph2.to_csv('./data/bokeh_monthly.csv',index=False)

In [ ]:
df_bokeh = pd.read_csv('./data/bokeh_monthly.csv')

In [ ]:
df_bokeh.sort_values(by=['year','month'],inplace=True)

In [ ]:
df_bokeh['month_year'] = tuple(zip(df_bokeh['month'], df_bokeh['year']))

df_bokeh['month_year'] = df_bokeh['month_year'].factorize()[0]

In [ ]:
df_bokeh.tail(100)

In [ ]:
df_bokeh.to_csv('./data/bokeh_monthly2.csv',index=False)

In [ ]:
df_bokeh = pd.read_csv('./data/bokeh_monthly2.csv')

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()
import os
from ipywidgets import interact
from bokeh.models import GMapOptions
from bokeh.plotting import gmap
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models import LinearInterpolator, CategoricalColorMapper,LinearColorMapper
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider
from bokeh.palettes import PuBu, Oranges

size_mapper = LinearInterpolator(x=[df_bokeh.wnvpresent.min(),df_bokeh.wnvpresent.max()],
                                y=[5,50])

color_mapper = LinearColorMapper(palette=Oranges[9],
                                 high=df_bokeh.wnvpresent.min(),
                                 low=df_bokeh.wnvpresent.max())

from ipywidgets import interact
from bokeh.io import push_notebook
#data1=ColumnDataSource(df_bokeh)

    #print('?')
    
#interact(update, m=(0,19,1))

map_options = GMapOptions(lat=41.881832, lng=-87.623177, map_type="roadmap", zoom=10)

# Replace the value below with your personal API key:
api_key = "****************************"


hover = HoverTool(
        tooltips=[
            ("Trap",'@trap'),
            ("Species",'@species'),
            ("Coordinates", "(@latitude{0.0000}, @longitude{0.0000})"),
        ]
    )

p = gmap(api_key, map_options, title="Chicago - West Nile Outbreak")

p.toolbar.active_inspect = [hover]

#data1=ColumnDataSource(df_bokeh)

data = dict(lat=df_bokeh.latitude.unique(),
            lon=df_bokeh.longitude.unique(),
            trap=df_bokeh.trap.unique(),
            species=df_bokeh.species)

#source=df_bokeh[df_bokeh.month_year==0]
source=df_bokeh

r = p.circle(x='longitude', y='latitude',
         size={'field':'wnvpresent','transform':size_mapper},
         fill_color={'field':'wnvpresent','transform':color_mapper},
         fill_alpha=0.6,
         source = df_bokeh[df_bokeh.month_year==5])
#p.circle(x="lon", y="lat", size=10, fill_color="blue", fill_alpha=0.6, source=data)
p.add_tools(hover)
#p.below=show(widgetbox(slider))
#show(widgetbox(slider))

#show(p,notebook_handle=True)

def update(m):
    #data = dict(lat=df_bokeh.loc[month_year].latitude,
    #        lon=df_bokeh.loc[month_year].longitude,
    #        trap=df_bokeh.loc[month_year].trap,
    #        species=df_bokeh.loc[month_year].species)
    #data1.data = data
    r.data_source.data['longitude']=df_bokeh[df_bokeh.month_year==m]['longitude']
    r.data_source.data['latitude']=df_bokeh[df_bokeh.month_year==m]['latitude']
    r.data_source.data['wnvpresent']=df_bokeh[df_bokeh.month_year==m]['wnvpresent']
    r.update
    hover = HoverTool(
        tooltips=[
            ("Trap",'@trap'),
            ("Species",'@species'),
            ("Coordinates", "(@latitude{0.0000}, @longitude{0.0000})"),
        ]
    )
    #p.add_tools(hover)
    
    #hover.update
    #print(m)
    #p.title.text="Chicago - West Nile Outbreak {}".format
    push_notebook()
    
interact(update, m=(0,19,1))
    
show(p,notebook_handle=True)

In [ ]:
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

In [ ]:
html = file_html(p, CDN, "my plot")

In [ ]:
Html_file= open("bokeh.html","w")
Html_file.write(html)
Html_file.close()


html

In [ ]:
 hover.tooltips[0][1]

In [ ]:
df_bokeh[df_bokeh.month_year==3]